In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from importlib import reload 

In [3]:
from deeprank.dataset import DataLoader, PairGenerator, ListGenerator

In [4]:
loader = DataLoader('./config/letor07_mp_fold1.model')

[./data/letor/r5w/word_dict.txt]
	Word dict size: 193367
[./data/letor/r5w/qid_query.txt]
	Data size: 1692
[./data/letor/r5w/docid_doc.txt]
	Data size: 65323
[./data/letor/r5w/embed_wiki-pdc_d50_norm]
	Embedding size: 109282
Generate numpy embed: (193368, 50)


In [14]:
import json
letor_config = json.loads(open('./config/letor07_mp_fold1.model').read())
device = torch.device("cuda")
# device = torch.device("cpu")

In [15]:
Letor07Path = letor_config['data_dir']

letor_config['fill_word'] = loader._PAD_
letor_config['embedding'] = loader.embedding
letor_config['feat_size'] = loader.feat_size
letor_config['vocab_size'] = loader.embedding.shape[0]
letor_config['embed_dim'] = loader.embedding.shape[1]

pair_gen = PairGenerator(rel_file=Letor07Path + '/relation.train.fold%d.txt'%(letor_config['fold']), 
                         config=letor_config)

[./data/letor/r5w/relation.train.fold1.txt]
	Instance size: 47828
Pair Instance Count: 325439


In [16]:
from deeprank import select_module
from deeprank import rank_module

select_module = reload(select_module)
rank_module = reload(rank_module)

In [17]:
select_net = select_module.IdentityNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)

In [18]:
# letor_config['simmat_channel'] = 1
# letor_config['conv_params'] = [(8, 2, 10)]
# letor_config['fc_params'] = [50]
# letor_config['dpool_size'] = [3, 10]
# letor_config['lr'] = 0.001
# letor_config['finetune_embed'] = False
# rank_net = rank_module.MatchPyramidNet(config=letor_config)
# rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
# rank_net.train()
# optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [19]:
letor_config['simmat_channel'] = 1
letor_config['conv_params'] = [(8, 3, 3)]
letor_config['fc_params'] = [200]
letor_config['dpool_size'] = [3, 10]
letor_config['lr'] = 0.001
letor_config['finetune_embed'] = False
rank_net = rank_module.MatchPyramidNet(config=letor_config)
rank_net = rank_net.to(device)
rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
rank_net.train()
optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [20]:
def to_device(*variables):
    return (torch.from_numpy(variable).to(device) for variable in variables)

In [21]:
import time
start_t = time.time()
for i in range(50):
    X1, X1_len, X2, X2_len, Y, F = pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
    X1, X1_len, X2, X2_len, Y, F = to_device(X1, X1_len, X2, X2_len, Y, F)
    X1, X2, X1_len, X2_len = select_net(X1, X2, X1_len, X2_len)
    output = rank_net(X1, X2, X1_len, X2_len)
    loss = rank_net.pair_loss(output, Y)
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
end_t = time.time()
print('Time Cost: %s s' % (end_t-start_t))

0.9819692373275757
0.9981075525283813
0.9330174922943115
0.9499410390853882
0.9110855460166931
1.0208740234375
0.9235810041427612
0.8712998032569885
0.8737620711326599
0.9343387484550476
0.9539638757705688
0.9646861553192139
0.8807889223098755
0.9769709706306458
0.7131575345993042
0.8897043466567993
1.0160967111587524
0.8744146823883057
0.9655374884605408
1.0145950317382812
0.7517421841621399
0.9148864150047302
0.6052004098892212
0.8982235193252563
0.9866613149642944
0.47787463665008545
0.5133614540100098
1.122140645980835
0.6825952529907227
0.6680004000663757
0.9706587195396423
1.0091679096221924
0.9361656308174133
0.8370518684387207
0.5855481028556824
0.7198920249938965
0.9910329580307007
1.0607331991195679
0.5471439361572266
0.5739837884902954
0.9599318504333496
0.9294826984405518
0.682215690612793
0.8660634756088257
0.44347941875457764
0.4878867566585541
0.6769955158233643
0.7865058183670044
1.4766498804092407
0.4539874792098999
Time Cost: 105.99338483810425 s


In [22]:
rank_net

MatchPyramidNet(
  (embedding): Embedding(193368, 50, padding_idx=0)
  (conv_sequential): Sequential(
    (0): Conv2d(1, 8, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  )
  (dpool_layer): AdaptiveMaxPool2d(output_size=[3, 10])
  (fc_sequential): Sequential(
    (0): Linear(in_features=240, out_features=200, bias=True)
  )
  (out_layer): Linear(in_features=200, out_features=1, bias=True)
)